In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import re
import unidecode
from wordcloud import WordCloud, STOPWORDS

In [3]:
df = pd.read_csv('trabajos.csv',encoding='latin-1')
df.head()

ParserError: Error tokenizing data. C error: Expected 10 fields in line 725, saw 24


In [ ]:
df.shape

In [ ]:
df.Id_Empleo.unique().shape

In [ ]:
df_new = df.drop_duplicates(subset ="Id_Empleo") 

In [ ]:
df_new.set_index('Id_Empleo', verify_integrity = True, inplace = True)

In [ ]:
df_new['Publicado'].value_counts()

In [ ]:
df_new['Sueldo'].value_counts()

In [ ]:
df_new['Nombre_Empresa'].value_counts().head(10)

In [ ]:
df_new['Ciudad'].value_counts()

In [ ]:
def column2text(feature):
    new_list = []
    # Separate by colon
    for element in feature:
        if ',' in element:
            tmp = element.split(',')
            new_list.append(tmp[0].rstrip())
            new_list.append(tmp[1].rstrip())
        else:
            new_categorias.append(element)
    # Merge all elements of list into a text        
    txt = ''
    for element in new_list:
        txt = txt + element + ' '
    # Clean text
    text = txt.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    #text = unidecode.unidecode(text) #Delete accents
    #text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    clean_text = text.replace('  ', ' ')
    return clean_text

In [ ]:
df_new['Nombre_cargo'].value_counts().head()

In [ ]:
temp = pd.DataFrame(new_categorias)
temp[0].value_counts().head()

In [ ]:
clean_text = column2text(categorias)

In [ ]:
wordcloud = WordCloud(
    collocations=False,
    width = 2000,
    height = 1500,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(clean_text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
nombres_cargos = column2text(df_new['Nombre_cargo'])

In [ ]:
#set the stopwords list
stopwords= set(STOPWORDS)

#append new words to the stopwords list
new_words = {'de', 'al', 'santa', 'cruz', 'cochabamba', 'la', 'paz'}
new_stopwords=stopwords.union(new_words)

wordcloud = WordCloud(
    collocations=False,
    width = 2000,
    height = 1500,
    background_color = 'black',
    stopwords = new_stopwords).generate(str(nombres_cargos))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

ver qué dias publican más publicidad?
en qué mes?
de los informaticos, cuantos son en banca?
eliminar stopwords espanol

If informatica:
    descripcion = 
    wordCloud(descripcion)